In [1]:
#Basic DataFrame & numerical libraries
import pandas as pd 

#Importing visualization libraries for exploratory analysis
import matplotlib.pyplot as plt 
import seaborn as sns 

#Importing to standardize formatting (geolocation)
import geopy

#Importing geocoder classes
from geopy.geocoders import GoogleV3

#Logical conclusion of importing pandas and geopy
import shapely #will help us work with geocoded data later
from shapely.geometry import Point, Polygon

#Importing necessary packages
import requests
from requests import get
import json
import re

#This library (safetyfile) contains a Google Maps API key.
#It is excluded from the uploaded dataset in the interest of informational security.
import safetyfile
from safetyfile import googleapi

print(type(googleapi))

<class 'str'>


In [2]:
#Reading original CSV to DataFrame
gtgarden = pd.read_csv('GreenThumb_Garden_Info_20240916.csv')
gtgarden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   assemblydist       624 non-null    int64  
 1   address            624 non-null    object 
 2   borough            624 non-null    object 
 3   communityboard     624 non-null    int64  
 4   congressionaldist  624 non-null    int64  
 5   coundist           624 non-null    int64  
 6   gardenname         624 non-null    object 
 7   juris              624 non-null    object 
 8   multipolygon       624 non-null    object 
 9   openhrsf           369 non-null    object 
 10  openhrsm           343 non-null    object 
 11  openhrssa          412 non-null    object 
 12  openhrssu          368 non-null    object 
 13  openhrsth          359 non-null    object 
 14  openhrstu          362 non-null    object 
 15  openhrsw           371 non-null    object 
 16  parksid            624 non

In [3]:
#Looking at a limited sample of entries
gtgarden.sample(5)

,assemblydist,address,borough,communityboard,congressionaldist,coundist,gardenname,juris,multipolygon,openhrsf,...,policeprecinct,statesenatedist,status,zipcode,BBL,NTA,CensusTract,lat,lon,crossStreets
181,32,"15012 115th Dr, Queens, NY 11434",Q,412,5,28,South Jamaica Infinity Garden,NYCHA,MULTIPOLYGON (((-73.79222562421799 40.68343954...,11:00 a.m. - 2:00 p.m.,...,113,10,Active,"11,434",4.121920e+09,/,NaN,NaN,NaN,155th Street & Sutphin Boulevard
539,54,1262 Madison Street,B,304,7,37,Madison Square Association Garden,DPR,MULTIPOLYGON (((-73.91312382400693 40.69488672...,10:30a - 3:00p,...,83,18,Active,"11,221",3.033700e+09,BK77 /,435.0,40.694823,-73.912965,Wilson & Knickerbocker
502,55,16 Somers Street,B,316,8,41,Phoenix Community Garden,DPR,MULTIPOLYGON (((-73.91119439985927 40.67866452...,CLOSED,...,73,25,Active,"11,233",3.015410e+09,BK79 /,371.0,40.678590,-73.912521,Corner of Fulton & Somers
371,56,862 Park Avenue Brooklyn NY 11206,B,303,8,36,Love Garden - Sumner Houses (NYCHA),NYCHA,MULTIPOLYGON (((-73.94066010924415 40.69849876...,NaN,...,79,25,Active,"11,206",3.015800e+09,/,NaN,NaN,NaN,Park Avenue & Marcus Garvey Blvd
279,68,52 East 117th Street,M,111,13,9,Peaceful Valley,DPR,MULTIPOLYGON (((-73.94404184045143 40.80036591...,CLOSED,...,25,30,Active,"10,035",1.016220e+09,MN34 /,184.0,40.800188,-73.943817,At Madison Avenue


In [4]:
#It looks like Pandas incorrectly read in ZIP Codes as floats...
#These function below should fix it.
def repairzip(textobj):
      return str(textobj).replace(',','')

In [5]:
#Let's put into action!
gtgarden['zipcode'] = gtgarden['zipcode'].apply(repairzip)
print(gtgarden['zipcode'].sample(5))

275    10009
40     10455
278    10025
43     10460
436    11221
Name: zipcode, dtype: object


In [6]:
#Finding coordinates problem entries, slicing into separate DataFrame
#We can use 'lat' as a proxy for both latitude and longitude: when one is absent, the other is absent

slice = gtgarden[pd.isnull(gtgarden['lat'])].copy()
slice.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 1 to 615
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   assemblydist       59 non-null     int64  
 1   address            59 non-null     object 
 2   borough            59 non-null     object 
 3   communityboard     59 non-null     int64  
 4   congressionaldist  59 non-null     int64  
 5   coundist           59 non-null     int64  
 6   gardenname         59 non-null     object 
 7   juris              59 non-null     object 
 8   multipolygon       59 non-null     object 
 9   openhrsf           25 non-null     object 
 10  openhrsm           24 non-null     object 
 11  openhrssa          27 non-null     object 
 12  openhrssu          23 non-null     object 
 13  openhrsth          25 non-null     object 
 14  openhrstu          24 non-null     object 
 15  openhrsw           25 non-null     object 
 16  parksid            59 non-null  

In [7]:
#Creating GoogleV3 class, searches using Google Map API to identify submitted addresses
#The aforementioned API key is used here.

geolocator = GoogleV3(api_key=googleapi)

In [8]:
#Using .apply() to basically create a Google Maps query for the address
#Some addresses lack building numbers: adding in the garden name AND ZIP Code gets around this problem
slice['pseudoaddress'] = slice.apply(lambda row: f'{row['gardenname']} {row['address']} {row['zipcode']}', axis = 1)

In [9]:
#Extracting geocodes relevant to each item...
slice['geocode'] = slice['pseudoaddress'].apply(lambda x: geolocator.geocode(x))

#This returns a geocode inherently incorporating both latitude and longitude
#On the off-chance a location is not on Google Maps, however, it might return 'None' instead

In [10]:
#And applying back as necessary...
def gc_lat(geocode):
    try:
        return geocode.latitude
    except AttributeError as err:
        return None
    
def gc_lon(geocode):
    try:
        return geocode.longitude
    except AttributeError as err:
        return None

slice['lat'] = slice['geocode'].apply(gc_lat).astype('float')
slice['lon'] = slice['geocode'].apply(gc_lon).astype('float')

In [11]:
#It turns out that there's a singular row in which the Google API was unable to determine its location...
#At index 130 is the "South Beach community garden NYCHA" at 100 Kramer street 10306.
#It's entirely unindexed by Google Maps. We do still have a standard address.
#We can clean this one up manually.

slice.loc[130, 'lat'] = gc_lat( geolocator.geocode(slice.loc[130, 'address']) )
slice.loc[130, 'lon'] = gc_lon( geolocator.geocode(slice.loc[130, 'address']) )

#Some other addresses only state the street name in ALL CAPS rather than the address.
#Google Maps, based on the provided information, is still able to approximate these locations.

In [12]:
#Let's drop the added column "pseudoaddress" now that we no longer need it...
slice = slice.drop(columns=['geocode'])

In [13]:
#With that done, let's now join this content back into the main DataFrame.
gtgarden.update(slice, overwrite=False, join='left', errors='ignore')
gtgarden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   assemblydist       624 non-null    int64  
 1   address            624 non-null    object 
 2   borough            624 non-null    object 
 3   communityboard     624 non-null    int64  
 4   congressionaldist  624 non-null    int64  
 5   coundist           624 non-null    int64  
 6   gardenname         624 non-null    object 
 7   juris              624 non-null    object 
 8   multipolygon       624 non-null    object 
 9   openhrsf           369 non-null    object 
 10  openhrsm           343 non-null    object 
 11  openhrssa          412 non-null    object 
 12  openhrssu          368 non-null    object 
 13  openhrsth          359 non-null    object 
 14  openhrstu          362 non-null    object 
 15  openhrsw           371 non-null    object 
 16  parksid            624 non

In [14]:
#Just to double check... no null values!
gtgarden[gtgarden['lon'].isnull()]

,assemblydist,address,borough,communityboard,congressionaldist,coundist,gardenname,juris,multipolygon,openhrsf,...,policeprecinct,statesenatedist,status,zipcode,BBL,NTA,CensusTract,lat,lon,crossStreets


In [15]:
#But we're seeing a problem with 'CensusTract'.
gtgarden[gtgarden['CensusTract'].isnull()].sample(5)

,assemblydist,address,borough,communityboard,congressionaldist,coundist,gardenname,juris,multipolygon,openhrsf,...,policeprecinct,statesenatedist,status,zipcode,BBL,NTA,CensusTract,lat,lon,crossStreets
366,57,"228 York Street, Brooklyn, NY 11201",B,302,7,35,Pulse Garden (NYCHA - Farragut Houses ),NYCHA,MULTIPOLYGON (((-73.98086416411832 40.70105330...,NaN,...,84,25,Active,11201,3.000710e+09,/,NaN,40.700646,-73.981580,York Street & Hudson Ave
485,60,786 Livonia Ave. Brooklyn,B,305,8,42,Fresh Farm,DPR,MULTIPOLYGON (((-73.88791904340812 40.66557304...,12:00 p.m. - 2:00 p.m.,...,75,19,Active,11207,3.040880e+09,/,NaN,40.665511,-73.887822,corner of Hendrix St. and Livonia Ave.
211,65,SPRING STREET,M,102,10,1,Elizabeth Street Garden,HPD,MULTIPOLYGON (((-73.99438387156921 40.72233956...,NaN,...,5,27,Not GreenThumb,10012,1.004930e+09,/,NaN,40.721316,-73.994710,btw Prince St. and Spring St.
497,46,SHELL ROAD,B,313,8,47,Victory Garden B13,DPR,MULTIPOLYGON (((-73.97480668585487 40.58264381...,NaN,...,60,23,Not GreenThumb,11224,3.072470e+09,/,NaN,40.582044,-73.975068,At Shore-Belt Pkwys
192,31,310 Beach 58th Street,Q,414,5,31,Rockaway Youth Community Power,DPR,MULTIPOLYGON (((-73.78879327802805 40.59468188...,8:00 a.m - Dusk,...,100,10,Active,11692,4.160040e+09,/,NaN,40.594537,-73.788083,at Beach Channel Drive


In [16]:
#While geopy doesn't have native support for US Census Geocoder API...
#A small package called 'censusgeocode' does.

import censusgeocode as cg

In [17]:
#Let's make another slice.
slice = gtgarden[gtgarden['CensusTract'].isnull()].copy()
slice.sample(5)['CensusTract']

368   NaN
110   NaN
370   NaN
360   NaN
148   NaN
Name: CensusTract, dtype: float64

In [18]:
#defining a function that can be used with apply
def extractcensustract(row):
    inlat = row['lat']
    inlon = row['lon']
    resultobj = cg.coordinates(x=inlon, y=inlat, returntype='geographies')
    tract = resultobj['Census Tracts'][0]['TRACT']
    tract = float(tract[:4]+"."+tract[4:])
    return tract

#Census tracts can either be expressed as a 6 digit code or as a float:
    #That is, tract 57.02 can be written as 005702 and vice versa.
    #For the purposes of this cleaning, we're converting all tracts into floats.
    #Actually, this makes them easier to find: most public resources use their float identity.

#Example
extractcensustract(slice.sample(1))

43.0

In [19]:
#Applying the function to the slice
slice['CensusTract'] = slice.apply(extractcensustract, axis=1)
slice[['address','CensusTract','lat','lon']].sample(5)

,address,CensusTract,lat,lon
372,201 Myrtle Avenue Brooklyn NY 11201,15.01,40.693527,-73.980010
100,438-44 Clarmont Pkwy,169.00,40.838943,-73.903770
368,"219 34th St. Brooklyn, NY 11232",84.00,40.654731,-74.001848
193,308 Beach 58th Street,954.00,40.592658,-73.797793
6,5240 Broadway Bronx NY 10463,309.00,40.875329,-73.908345


In [20]:
#Return again to the main DataFrame!
gtgarden.update(slice, overwrite=False, join='left', errors='ignore')
gtgarden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   assemblydist       624 non-null    int64  
 1   address            624 non-null    object 
 2   borough            624 non-null    object 
 3   communityboard     624 non-null    int64  
 4   congressionaldist  624 non-null    int64  
 5   coundist           624 non-null    int64  
 6   gardenname         624 non-null    object 
 7   juris              624 non-null    object 
 8   multipolygon       624 non-null    object 
 9   openhrsf           369 non-null    object 
 10  openhrsm           343 non-null    object 
 11  openhrssa          412 non-null    object 
 12  openhrssu          368 non-null    object 
 13  openhrsth          359 non-null    object 
 14  openhrstu          362 non-null    object 
 15  openhrsw           371 non-null    object 
 16  parksid            624 non

In [21]:
#We see that some results still lack crossStreets: that is, intersections.
slice = gtgarden[gtgarden['crossStreets'].isna()].copy()
print(slice.sample(5))

#Unfortunately, Google's API doesn't support returning intersections.
#In some cases, identifiying intersections might be inappropriate.
#Given that we're cleaning this dataset for later visualization, this column isn't essential for user use.

#For now, we'll fill these with the string value 'N/A'.
#These can be updated with new values from an updated version of the sheet.

gtgarden['crossStreets'] = gtgarden['crossStreets'].fillna('N/A')

#We can, however, address some shorthand which might not show up well in our ultimate visualization.
gtgarden['crossStreets'] = gtgarden['crossStreets'].replace(r'[Bb][Tt][Ww][Nn]?', r'Between', regex=True)

     assemblydist                                  address borough  \
360            53      50 Humbold street Brooklyn NY 11206       B   
148            30        41-38 69th St, Flushing, NY 11377       Q   
142            30  31-57 51st St, Woodside, New York 11377       Q   
195            29                      106-17 173rd Street       Q   
168            32     12001 142nd Place, South Ozone Parks       Q   

     communityboard  congressionaldist  coundist  \
360             301                  7        34   
148             402                  6        26   
142             401                  7        25   
195             412                  5        27   
168             412                  5        28   

                                   gardenname  juris  \
360  Bushwick Garden- Bushwick houses (NYCHA)  NYCHA   
148                   Queens New Roots (East)    DOT   
142             Moore Jackson Cemetery/Garden    PRI   
195     Liberty Collective Learning Garden

In [22]:
#It still looks like we have some blank values here and there...
gtgarden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   assemblydist       624 non-null    int64  
 1   address            624 non-null    object 
 2   borough            624 non-null    object 
 3   communityboard     624 non-null    int64  
 4   congressionaldist  624 non-null    int64  
 5   coundist           624 non-null    int64  
 6   gardenname         624 non-null    object 
 7   juris              624 non-null    object 
 8   multipolygon       624 non-null    object 
 9   openhrsf           369 non-null    object 
 10  openhrsm           343 non-null    object 
 11  openhrssa          412 non-null    object 
 12  openhrssu          368 non-null    object 
 13  openhrsth          359 non-null    object 
 14  openhrstu          362 non-null    object 
 15  openhrsw           371 non-null    object 
 16  parksid            624 non

In [23]:
#Column indices [9,15] are all describing open hours.
#A bit confusingly, they go in the order of: [Friday, Monday, Saturday, Sunday, Thursday, Tuesday, Wednesday].
#We can conver this to [Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday] at a later point.

slice = gtgarden.iloc[:,9:16].copy()
slice.sample(15)

,openhrsf,openhrsm,openhrssa,openhrssu,openhrsth,openhrstu,openhrsw
524,NaN,9:00 a.m. - 11:00 a.m.,8:00 a.m. - 11:00 p.m.,NaN,NaN,NaN,8:00 a.m. - 11:00 a.m.
148,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,CLOSED,CLOSED,9:00 a.m. - 7:00 p.m.,9:00 a.m. - 7:00 p.m.,CLOSED,CLOSED,CLOSED
472,CLOSED,CLOSED,10:00 a.m. - 6:00 p.m.,10:00 a.m. - 6:00 p.m.,CLOSED,CLOSED,10:00 a.m. - 2:00 p.m.
398,9:00a - 2:00p,2:00p - 6:00p,10:00a - 5:00p,10:00a - 5:00p,9:00a - 2:00p,2:00p - 6:00p,2:00p - 6:00p
330,9:00 a.m.- 3:00 p.m.,9:00 a.m.- 3:00 p.m.,9:00 a.m.- 3:00 p.m.,9:00 a.m.- 3:00 p.m.,9:00 a.m.- 3:00 p.m.,9:00 a.m.- 3:00 p.m.,9:00 a.m.- 3:00 p.m.
224,NaN,NaN,12:00p - 5:00p,12:00p - 5:00p,12:00p - 5:00p,NaN,NaN
559,9:00 a.m. - 12:00 p.m.,9:00 a.m. - 12:00 p.m.,9:00 a.m. - 12:00 p.m.,9:00 a.m. - 12:00 p.m.,9:00 a.m. - 12:00 p.m.,9:00 a.m. - 12:00 p.m.,9:00 a.m. - 12:00 p.m.
176,NaN,NaN,10:00a - 2:00p,NaN,2:00p - 8:00p,6:00p - 8:00p,NaN
196,NaN,12:00 p.m. - 1:00 p.m.,9:00 a.m. -1:00 p.m.,"9:00 a.m. - 12:00 p.m. & 3:00 p.m,. -5:00 p.m.",NaN,12:00 p.m. - 1:00 p.m.,2:00 p.m. - 3:00 p.m.


In [24]:
#There seem to be some tiny errors...
slice[slice=='close'].count()

#For example, instances of having written "close" or "Close" or even "Closed" as opposed to standard "CLOSED".
#This is an easy fix.

openhrsf     0
openhrsm     0
openhrssa    1
openhrssu    2
openhrsth    0
openhrstu    1
openhrsw     2
dtype: int64

In [25]:
#This regex searches for any variation on "CLOSED", D-optional, and replace them with "CLOSED".
slice = slice.replace(r'[Cc][Ll][Oo][Ss][Ee][dD]?','CLOSED', regex=True)

In [26]:
#There's still tiny inconsistencies like how some entries include "a" or "p" in place of "a.m." or "p.m."
#We can also fix that easily.

slice.sample(15)

,openhrsf,openhrsm,openhrssa,openhrssu,openhrsth,openhrstu,openhrsw
290,NaN,NaN,NaN,9:00 a.m. - 7:00 p.m.,NaN,NaN,NaN
165,8:00 a.m. - 12:00 p.m.,8:00 a.m. - 10:30 a.m.,9:00 a.m. - 3:00 p.m.,CLOSED,CLOSED,CLOSED,9:30 a.m. - 11:30 a.m.
358,10:00 a.m. - 2:00 p.m.,NaN,10:00 a.m. - 2:00 p.m.,NaN,10:00 a.m. - 2:00 p.m.,10:00 a.m. - 2:00 p.m.,10:00 a.m. - 2:00 p.m.
201,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,11:30 a.m. - 5:30 p.m.,CLOSED,12:00 p.m. - 6:00 p.m.,CLOSEDa,11:30 a.m. - 5:30 p.m.,11:30 a.m. - 5:30 p.m.,11:30 a.m. - 5:30 p.m.
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,8:00 a.m. - 8:00 p.m.,8:00 a.m. - 8:00 p.m.,8:00 a.m. - 8:00 p.m.,8:00 a.m. - 8:00 p.m.,8:00 a.m. - 8:00 p.m.,8:00 a.m. - 8:00 p.m.,8:00 a.m. - 8:00 p.m.
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,10:00a - 2:00p,NaN,2:00p - 8:00p,6:00p - 8:00p,NaN


In [27]:
#Correcting single character formatting
slice = slice.replace(r'(\d*:?\d*)([Aa])(\s)','\\1 a.m.\\3',regex=True)
slice = slice.replace(r'(\d*:?\d*)([Aa])(\s?$)','\\1 a.m.\\3',regex=True)
slice = slice.replace(r'(\d*:?\d*)([Pp])(\s)','\\1 p.m.\\3',regex=True)
slice = slice.replace(r'(\d*:?\d*)([Pp])(\s?$)','\\1 p.m.\\3',regex=True)

#Eliminating inconsistent spacing and stray numerals
slice = slice.replace(r'([1-9])(:)([Pp])','\\1:00 \\3',regex=True)
slice = slice.replace(r'([1-9])(:)([Aa])','\\1:00 \\3',regex=True)
slice = slice.replace(r'^([1-9]?[1-9])(:)?\s*([Pp])','\\1:00 \\3',regex=True)
slice = slice.replace(r'^([1-9]?[1-9])(:)?\s*([Aa])','\\1:00 \\3',regex=True)

#General consistency
slice = slice.replace(r'[Aa].?[Mm].?','a.m.', regex=True)
slice = slice.replace(r'[Pp].?[Mm].?','p.m.', regex=True)
slice = slice.replace(r'-','to', regex=True)
slice = slice.replace(r'Noon','12:00 p.m.', regex=True)
slice = slice.replace(r'.-', ' -', regex=True)
slice = slice.replace(r'\s?(:)\s?',':',regex=True)
slice = slice.replace(r'(\w)(to)(\w)',r'\1 to \3', regex=True)


#Dealing with lists
slice = slice.replace(r'\s?(,|&|;)(\s*)(\d)',r';\n\3', regex=True)


slice.sample(15)

,openhrsf,openhrsm,openhrssa,openhrssu,openhrsth,openhrstu,openhrsw
286,CLOSED,CLOSED,11:00 a.m. to 4:00 p.m.,11:00 a.m. to 4:00 p.m.,CLOSED,CLOSED,CLOSED
571,12:00 p.m. to 3:00 p.m.,10:00 a.m. to 1:00 p.m.,3:00 p.m. to 4:30 p.m.,3:00 p.m. to 5:30 p.m.,10:00 a.m. to 1:00 p.m.,12:00 p.m. to 3:00 p.m.,3:00 p.m. to 6:00 p.m.
260,NaN,NaN,11:00 a.m. to 4:00 p.m.,12:00 a.m. to 5:00 p.m.,2:00 p.m. to 7:00 p.m.,1:00 p.m. to 6:00 p.m.,NaN
528,9:00 a.m. to 11:30 a.m.,9:00 a.m. to 11:30 a.m.,10:00 p.m. to 3:00 p.m.,NaN,9:00 a.m. to 11:30 a.m.,9:00 a.m. to 11:30 a.m.,9:00 a.m. to 11:30 a.m.
429,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520,CLOSED,CLOSED,CLOSED,8:00 a.m.to 1:00 p.m.,8:00 a.m. to 5:00 p.m.,CLOSED,8:00 a.m. to 5:00 p.m.
68,9:00 a.m. to 5:00 p.m.,9:00 a.m. to 5:00 p.m.,9:00 a.m. to 5:00 p.m.,9:00 a.m. to 5:00 p.m.,9:00 a.m. to 5:00 p.m.,9:00 a.m. to 5:00 p.m.,9:00 a.m. to 5:00 p.m.
295,CLOSED,CLOSED,12:00 p.m. to 5:00 p.m.,12:00 p.m. to 5:00 p.m.,CLOSED,5:00 p.m. to 7:00 p.m.,CLOSED
156,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#Unlike latitude or longitude, we can't extrapolate other information to fill these times.
#It's probably not appropriate to assume that they're closed during unlisted times either...

#Pending further updates on the original sheet, we can fill these with a 'N/A' label.

#There's data-original oddities like 'Sunset to Sundown' at '955 Columbus Avenue'.
#It might best to leave these alone: again, there's not other information to extrapolate from for proper corrections.

slice = slice.fillna('N/A')
slice.sample(15)

,openhrsf,openhrsm,openhrssa,openhrssu,openhrsth,openhrstu,openhrsw
551,12:00 p.m. to 5:00 p.m.,12:00 p.m. to 4:00 p.m.,1:00 p.m. to 6:00 p.m.,1:00 p.m. to 6:00 p.m.,N/A,N/A,12:00 p.m. to 4:00 p.m.
24,10:00 a.m. to 1:00 p.m.,N/A,12:00 p.m. to 5:00 p.m.,12:00 p.m. to 05:00 p.m.,4:00 p.m. to 7:00 p.m.,4:00 p.m. to 7:00 p.m.,4:00 p.m. to 7:00 p.m.
357,N/A,N/A,N/A,N/A,N/A,N/A,N/A
249,10:00 a.m. to 2:00 p.m.,N/A,10:00 a.m. to 2:00 p.m.,N/A,10:00 a.m. to 2:00 p.m.,10:00 a.m. to 2:00 p.m.,10:00 a.m. to 2:00 p.m.
324,9:00 a.m. to 4:00 p.m.,9:00 a.m. to 12:00 p.m.,12:00 p.m. to 7:00 p.m.,7:00 a.m. to 12:00 p.m.,9:00 a.m. to 4:00 p.m.,9:00 a.m. to 4:00 p.m.,9:00 a.m. to 4:00 p.m.
149,N/A,N/A,N/A,N/A,N/A,N/A,N/A
148,N/A,N/A,N/A,N/A,N/A,N/A,N/A
106,10:00 a.m. to 6:00 p.m.,12:00 p.m. to 6:00 p.m.,10:00 a.m. to 6:00 p.m.,11:00 a.m. to 6:00 p.m.,10:00 a.m. to 6:00 p.m.,10:00 a.m. to 6:00 p.m.,10:00 a.m. to 6:00 p.m.
409,N/A,N/A,N/A,N/A,N/A,N/A,N/A
437,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [29]:
#Back to the main DataFrame.
gtgarden.update(slice, overwrite=True, join='left', errors='ignore')
gtgarden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   assemblydist       624 non-null    int64  
 1   address            624 non-null    object 
 2   borough            624 non-null    object 
 3   communityboard     624 non-null    int64  
 4   congressionaldist  624 non-null    int64  
 5   coundist           624 non-null    int64  
 6   gardenname         624 non-null    object 
 7   juris              624 non-null    object 
 8   multipolygon       624 non-null    object 
 9   openhrsf           624 non-null    object 
 10  openhrsm           624 non-null    object 
 11  openhrssa          624 non-null    object 
 12  openhrssu          624 non-null    object 
 13  openhrsth          624 non-null    object 
 14  openhrstu          624 non-null    object 
 15  openhrsw           624 non-null    object 
 16  parksid            624 non

In [30]:
#Nice! Now let's clear away non-necessities...
gtgarden = gtgarden.map(lambda x: x.strip() if isinstance(x, str) else x)
gtgarden.sample(5)

,assemblydist,address,borough,communityboard,congressionaldist,coundist,gardenname,juris,multipolygon,openhrsf,...,policeprecinct,statesenatedist,status,zipcode,BBL,NTA,CensusTract,lat,lon,crossStreets
131,63,97 Myrtle Avenue,R,501,11,49,Olivet Heavenly Harvest,PRI,MULTIPOLYGON (((-74.11620321226985 40.63045721...,N/A,...,120,23,Active,10310,5.001820e+09,SI35 /,105.0,40.630199,-74.115963,N/A
464,56,411 Herkimer Street,B,303,8,36,Her-King Alagantic Community Garden,DPR,MULTIPOLYGON (((-73.93970334877692 40.67918341...,10:00 a.m. to 2:00p.m.;\n5:00 p.m. to 7:00 p.m.,...,79,20,Active,11213,3.018640e+09,BK61 /,271.0,40.679318,-73.939658,Kingston Ave. & Albany Ave.
40,84,360 E. 151st Street,X,201,15,17,Neighborhood Advisory Committee Community Garden,BLT,MULTIPOLYGON (((-73.91831706203502 40.81774807...,N/A,...,40,29,Active,10455,2.023970e+09,BX34 /,65.0,40.817613,-73.918473,Courtlandt & Melrose Avenues
502,55,16 Somers Street,B,316,8,41,Phoenix Community Garden,DPR,MULTIPOLYGON (((-73.91119439985927 40.67866452...,CLOSED,...,73,25,Active,11233,3.015410e+09,BK79 /,371.0,40.678590,-73.912521,Corner of Fulton & Somers
415,56,87 Schenectady Avenue,B,308,9,36,Imani Community Garden (Schenectady),NYRP,MULTIPOLYGON (((-73.93308062412206 40.67633922...,N/A,...,77,20,Active,11213,3.013420e+09,BK61 /,309.0,40.676314,-73.932890,Between Pacific St. and Dean St.


In [31]:
#Let's sort the columns into a more logical order.
#We'll prioritize unique information, like name, address, and coordinates first.
#Status will also be prioritized.

#More categorical tags, like congressional districts, can be moved after them.
#We'll move open hours to the very back...

gtgarden = gtgarden[['parksid',
                    'gardenname',
                    'status',
                    'address',
                    'lat',
                    'lon',
                    'BBL',
                    'borough',
                    'crossStreets',
                    'zipcode',
                    'openhrsm',
                    'openhrstu',
                    'openhrsw',
                    'openhrsth',
                    'openhrsf',
                    'openhrssa',
                    'openhrssu',
                    'CensusTract',
                    'assemblydist',
                    'communityboard',
                    'NTA',
                    'congressionaldist',
                    'coundist',
                    'statesenatedist',
                    'policeprecinct',
                    'juris',
                    'multipolygon']]

gtgarden.sample(5)

,parksid,gardenname,status,address,lat,lon,BBL,borough,crossStreets,zipcode,...,CensusTract,assemblydist,communityboard,NTA,congressionaldist,coundist,statesenatedist,policeprecinct,juris,multipolygon
555,B465-GT001,First Quincy Street Community Garden,Active,397 Quincy Street,40.687912,-73.944384,3.018050e+09,B,Thompkins & Throop Avenues,11221,...,265.00,56,303,BK75 /,8,36,25,79,DPR,MULTIPOLYGON (((-73.9442475190063 40.688071055...
277,M370-GT001,Sam & Sadie Koenig Garden,Active,237 E. 7th Street,40.724094,-73.978343,1.003770e+09,M,Avenues C & D,10009,...,26.02,74,103,MN28 /,10,2,27,9,DPR,MULTIPOLYGON (((-73.97837193287097 40.72412198...
319,M325-GT003,La Casita Garden,Active,339 East 8th Street,40.725255,-73.978584,1.003910e+09,M,Ave. B and Ave. C,10009,...,26.02,74,103,MN28 /,10,2,27,9,DPR,MULTIPOLYGON (((-73.97855546935693 40.72514310...
288,M356-GT001,Harlem Roots Community Garden,Active,203 W. 120th Street,40.806224,-73.951109,1.019260e+09,M,Adam C. Powell Blvd. & Street Nicholas Avenue,10027,...,220.00,70,110,MN11 /,13,9,30,28,DPR,MULTIPOLYGON (((-73.95097672415767 40.80632973...
342,M298-GT001,Unity Park,Active,55 West 128th Street,40.809341,-73.942661,1.017260e+09,M,5th Ave. & Lenox Ave.,10027,...,208.00,70,110,MN03 /,13,9,30,32,DPR,MULTIPOLYGON (((-73.94248043243698 40.80942764...


In [32]:
#It's really weird that the boroughs are acronymized in this way...
#The good thing is that the creators of this dataset made every borough have a unique one-character symbol.
#We'll replace them with the function below:

def boroughsort(chara):
    if chara == 'M':
        return 'Manhattan'
    elif chara == 'X':
        return 'Bronx'
    elif chara == 'B':
        return 'Brooklyn'
    elif chara == 'Q':
        return 'Queens'
    else:
        return 'Staten Island'
    
gtgarden['borough'] = gtgarden['borough'].apply(boroughsort)
gtgarden['borough'].sample(5)

142       Queens
475     Brooklyn
353    Manhattan
433     Brooklyn
302    Manhattan
Name: borough, dtype: object

In [33]:
#These are corrections to a few... small unique errors in the original dataset.
#For example, this garden in the Bronx being des.ignated as Brooklyn.

print(gtgarden.loc[5,'gardenname'])
print(gtgarden.loc[5,'borough'])
gtgarden.loc[5, 'borough'] = 'Bronx'

#Easy fix.
#We've made this dataset usable, but it might take some more work than this to make it perfect.

People Garden- Patterson Houses (NYCHA)
Brooklyn


In [34]:
#It'll be nice if we can sort these by neighborhood.
#These records include NTA (Neighborhood Tabulation Areas), but unfortunately these only contain the serial codes.
#Some of these are still missing, too... The ones present are based on the 2010 NTAs.
#First, let's import the NTA table as a DataFrame.

ntatableraw = get("https://data.cityofnewyork.us/resource/q2z5-ai38.json").json()
ntarecords = pd.DataFrame(ntatableraw)
ntarecords.info()

ntarecords.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   the_geom     195 non-null    object
 1   boro_code    195 non-null    object
 2   boro_name    195 non-null    object
 3   county_fips  195 non-null    object
 4   ntacode      195 non-null    object
 5   ntaname      195 non-null    object
 6   shape_leng   195 non-null    object
 7   shape_area   195 non-null    object
dtypes: object(8)
memory usage: 12.3+ KB


,the_geom,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area
51,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",1,Manhattan,061,MN21,Gramercy,12096.890667,7526916.50304
134,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",2,Bronx,005,BX62,Woodlawn-Wakefield,38692.782777,39738653.3155
188,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",3,Brooklyn,047,BK68,Fort Greene,19825.3288358,16482758.9218
104,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",3,Brooklyn,047,BK40,Windsor Terrace,19033.6715536,14041667.8647
169,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",2,Bronx,005,BX40,Fordham South,15880.3526191,6308874.98599


In [35]:
#Let's transform those geometries into multipolygon shapes.
#What we're trying to do is check if these points are in these multipolygons...
ntarecords['the_geom'] = ntarecords['the_geom'].apply(lambda x: dict(x))
ntarecords['the_geom'] = ntarecords['the_geom'].apply(lambda x: shapely.geometry.shape(x))
ntarecords['the_geom'].sample(4)

#Great!

123    MULTIPOLYGON (((-73.79303800052352 40.71071107...
145    MULTIPOLYGON (((-74.08373397144521 40.63573714...
169    MULTIPOLYGON (((-73.89138023380261 40.86170058...
29     MULTIPOLYGON (((-73.96514385192519 40.59110191...
Name: the_geom, dtype: object

In [36]:
#Cool! Now let's clean the NTA column in 'gtgarden'.

gtgarden['NTA'] = gtgarden['NTA'].apply(lambda x: x[:4])
gtgarden['NTA'].sample(3)

17     BX14
155    QN12
167       /
Name: NTA, dtype: object

In [37]:
#We still have to take care of those empty 'NTA' rows...
#The 'slice' is back!

slice = gtgarden[gtgarden['NTA'] == '/'].copy()
slice.sample(5)

,parksid,gardenname,status,address,lat,lon,BBL,borough,crossStreets,zipcode,...,CensusTract,assemblydist,communityboard,NTA,congressionaldist,coundist,statesenatedist,policeprecinct,juris,multipolygon
147,QGT026,Queens New Roots (West),Active,"41-38 69th St, Flushing, NY 11377",40.743327,-73.895850,4.013090e+09,Queens,N/A,11377,...,265.02,30,402,/,6,26,12,108,DOT,MULTIPOLYGON (((-73.89586070593516 40.74313564...
146,QGT027,Serinor Community Garden - International Tower...,Active,"90-20 170th Street, Queens 11432",40.706260,-73.788923,4.098020e+09,Queens,170th street & 90th Ave,11432,...,444.00,29,412,/,5,27,11,103,NYCHA,MULTIPOLYGON (((-73.79110972642091 40.70848436...
469,B562-GT001,Surfside Garden Multi-Cultural Coalition,Active,2871 Surf Avenue,40.573371,-73.994704,3.070520e+09,Brooklyn,At W. 29th Street between Surf and Mermaid,11224,...,342.00,46,313,/,8,47,23,60,DPR,MULTIPOLYGON (((-73.99459582875232 40.57306887...
7,XGT084,Botanical Square Community Garden,Not GreenThumb,"419 Botanical Square S, The Bronx, NY 10",40.867219,-73.881742,2.032740e+09,Bronx,N/A,10458,...,334.00,78,207,/,15,11,33,52,MTA,MULTIPOLYGON (((-73.88217370142452 40.86732503...
203,MGT098,Liberty Community Gardens (LCG),Not GreenThumb,"200-218 Albany St, New York, NY 10280",40.709640,-74.015419,NaN,Manhattan,N/A,10280,...,317.04,61,101,/,10,1,27,1,PRI,MULTIPOLYGON (((-74.01544203984173 40.70976340...


In [38]:
#For our own ease, let's make some "Points" in this isolated DataFrame
slice['Points'] = slice.apply(lambda row: Point(row['lon'],row['lat']), axis=1)
slice['Points'].sample(5)

360    POINT (-73.9403351 40.7030605)
146     POINT (-73.788923 40.7062604)
121    POINT (-73.9094009 40.8202996)
456     POINT (-73.9229704 40.684711)
100    POINT (-73.9037701 40.8389431)
Name: Points, dtype: object

In [39]:
#Now, let's try and apply this...

def find_NTA(point):
    for instance, row in ntarecords.iterrows():
        if row['the_geom'].contains(point):
            return row['ntacode']
    return None

slice['NTA'] = slice['Points'].apply(lambda x: find_NTA(x))
slice['NTA'].sample(5)

#Success!

132    SI37
497    BK21
100    BX01
371    BK35
359    BK38
Name: NTA, dtype: object

In [40]:
#Let's put things back where they were.
gtgarden.update(slice, overwrite=True, join='left', errors='ignore')
gtgarden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   parksid            624 non-null    object 
 1   gardenname         624 non-null    object 
 2   status             624 non-null    object 
 3   address            624 non-null    object 
 4   lat                624 non-null    float64
 5   lon                624 non-null    float64
 6   BBL                622 non-null    float64
 7   borough            624 non-null    object 
 8   crossStreets       624 non-null    object 
 9   zipcode            624 non-null    object 
 10  openhrsm           624 non-null    object 
 11  openhrstu          624 non-null    object 
 12  openhrsw           624 non-null    object 
 13  openhrsth          624 non-null    object 
 14  openhrsf           624 non-null    object 
 15  openhrssa          624 non-null    object 
 16  openhrssu          624 non

In [41]:
#Let's crosscheck these objects again.
def nbcrosscheck(nta):
    for instance, row in ntarecords.iterrows():
        if row['ntacode'] == nta:
            return row['ntaname'] #Essentially the same as checking which polygons fell where
            break
    return None

gtgarden['neighborhood'] = gtgarden['NTA'].apply(nbcrosscheck)
gtgarden['neighborhood'].sample(5)

198    Hunters Point-Sunnyside-West Maspeth
418                             Brownsville
528                           East New York
404                      Park Slope-Gowanus
507                             Brownsville
Name: neighborhood, dtype: object

In [42]:
#Let's put things back in order.
gtgarden = gtgarden[['parksid',
                    'gardenname',
                    'status',
                    'address',
                    'lat',
                    'lon',
                    'BBL',
                    'neighborhood',
                    'borough',
                    'crossStreets',
                    'zipcode',
                    'openhrsm',
                    'openhrstu',
                    'openhrsw',
                    'openhrsth',
                    'openhrsf',
                    'openhrssa',
                    'openhrssu',
                    'CensusTract',
                    'assemblydist',
                    'communityboard',
                    'NTA',
                    'congressionaldist',
                    'coundist',
                    'statesenatedist',
                    'policeprecinct',
                    'juris',
                    'multipolygon']]

In [43]:
#Great! Now let's also rename these so they're more useful for us.
#First, let's set everything to CamelCase. We'll also unshorten and deacronymize names.
#In the case of the hours, we'll rename them for the sake of clarity.

gtgarden.rename(columns={   'parksid':'ParkID',
                    'gardenname':'GardenName',
                    'status':'Status',
                    'address':'Address',
                    'lat':'Latitude',
                    'lon':'Longitude',
                    'BBL':'BoroughBlockLot',
                    'neighborhood':'Neighborhood',
                    'borough':'Borough',
                    'crossStreets':'CrossStreets',
                    'zipcode':'ZIPCode',
                    'openhrsm':'MondayHours',
                    'openhrstu':'TuesdayHours',
                    'openhrsw':'WednesdayHours',
                    'openhrsth':'ThursdayHours',
                    'openhrsf':'FridayHours',
                    'openhrssa':'SaturdayHours',
                    'openhrssu':'SundayHours',
                    'CensusTract':'CensusTract',
                    'assemblydist':'AssemblyDistrict',
                    'communityboard':'CommunityBoard',
                    'NTA':'NeighborhoodTabulationArea',
                    'congressionaldist':'CongressionalDistrict',
                    'coundist':'CityCouncilDistrict',
                    'statesenatedist':'StateSenateDistrict',
                    'policeprecinct':'PolicePrecinct',
                    'juris':'Jurisdiction',
                    'multipolygon':'MultipolygonShape'
                },  
                inplace=True    )

gtgarden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ParkID                      624 non-null    object 
 1   GardenName                  624 non-null    object 
 2   Status                      624 non-null    object 
 3   Address                     624 non-null    object 
 4   Latitude                    624 non-null    float64
 5   Longitude                   624 non-null    float64
 6   BoroughBlockLot             622 non-null    float64
 7   Neighborhood                624 non-null    object 
 8   Borough                     624 non-null    object 
 9   CrossStreets                624 non-null    object 
 10  ZIPCode                     624 non-null    object 
 11  MondayHours                 624 non-null    object 
 12  TuesdayHours                624 non-null    object 
 13  WednesdayHours              624 non

In [44]:
#One last thing...
#Let's standardize street names in the addresses.
#Specifically: we want to target those capitalizations.

def titlecase(address):
    return re.sub(r'([A-Z]{3,})', lambda x: x.group(0).title(), address) 

#This one was a little hard to figure out
#Specifically using 3 or more here to make sure naming conventions with successive capital letters are unaffected
#Didn't use blanket .title() method to respect Irish names (among others)

gtgarden['Address'] = gtgarden['Address'].apply(titlecase)

In [45]:
#This seems good enough to go!
#Let's output our new, cleaned, upgraded dataset.

gtgarden_postclean = gtgarden

In [46]:
#Write cleaned DataFrame to CSV!
gtgarden_postclean.to_csv("greenthumb_garden_clean.csv", sep=',', encoding='utf-8', index=False)